In [1]:
# load standard libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
# load the data of manchester police
raw_data = pd.read_pickle('Manchester_street.pickle')
N = len(raw_data)

In [3]:
# summary info on the data
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3034429 entries, 0 to 3034428
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Crime ID               object 
 1   Month                  object 
 2   Reported by            object 
 3   Falls within           object 
 4   Longitude              float64
 5   Latitude               float64
 6   Location               object 
 7   LSOA code              object 
 8   LSOA name              object 
 9   Crime type             object 
 10  Last outcome category  object 
 11  Context                object 
dtypes: float64(2), object(10)
memory usage: 277.8+ MB


In [4]:
len(raw_data['Crime ID'].unique()) / N

# conclusion ~33% of crimes have no ID

0.6621357757917552

In [5]:
# replace the Month column, with integer values
raw_data['year'] = pd.DatetimeIndex(raw_data['Month']).year
raw_data['month'] = pd.DatetimeIndex(raw_data['Month']).month
raw_data.drop('Month', axis=1, inplace=True)

# check these two columns, as they are the same we can delete them, as they can't influence the outcome prediction
raw_data['Reported by'].unique(), raw_data['Falls within'].unique()

# we have multiple variables that give location data, location as a column is non specific and needs to be encoded on top to be worked with
# therefore we can drop the column
raw_data.drop('Location', axis=1, inplace=True)

In [6]:
# # short tests with LSOA codes/names
# code = 'E01032485'
# raw_data[raw_data['LSOA code'] == code]['LSOA name'].unique()

In [7]:
len(raw_data['LSOA code'].unique()), len(raw_data['LSOA name'].unique())
# for now we will assume that they are identical and we can drop one column, this needs to be further investigated
raw_data.drop('LSOA code', axis=1, inplace=True)

In [8]:
# one third of the dataset does not have an outcome category, as we are currently NOT specifying crime type that
# importantly we can drop the column now, but if we think crime type is very important, we will have to add it back in, 
# for more nuance we would also have to further look into the nan values then
raw_data[raw_data['Last outcome category'].isna()]

,Crime ID,Reported by,Falls within,Longitude,Latitude,LSOA name,Crime type,Last outcome category,Context,year,month
0,NaN,Greater Manchester Police,Greater Manchester Police,-2.392346,53.634640,Blackburn with Darwen 018E,Anti-social behaviour,NaN,NaN,2010,12
1,NaN,Greater Manchester Police,Greater Manchester Police,-2.444807,53.611509,Bolton 001A,Anti-social behaviour,NaN,NaN,2010,12
2,NaN,Greater Manchester Police,Greater Manchester Police,-2.444807,53.611509,Bolton 001A,Burglary,NaN,NaN,2010,12
3,NaN,Greater Manchester Police,Greater Manchester Police,-2.440493,53.612388,Bolton 001A,Vehicle crime,NaN,NaN,2010,12
4,NaN,Greater Manchester Police,Greater Manchester Police,-2.444807,53.611509,Bolton 001A,Vehicle crime,NaN,NaN,2010,12
...,...,...,...,...,...,...,...,...,...,...,...
3034407,NaN,Greater Manchester Police,Greater Manchester Police,-2.564641,53.474504,Wigan 040D,Anti-social behaviour,NaN,NaN,2019,6
3034408,NaN,Greater Manchester Police,Greater Manchester Police,-2.564641,53.474504,Wigan 040D,Anti-social behaviour,NaN,NaN,2019,6
3034409,NaN,Greater Manchester Police,Greater Manchester Police,-2.560912,53.475070,Wigan 040D,Anti-social behaviour,NaN,NaN,2019,6
3034410,NaN,Greater Manchester Police,Greater Manchester Police,-2.564641,53.474504,Wigan 040D,Anti-social behaviour,NaN,NaN,2019,6


In [9]:
raw_data.drop('Last outcome category', axis=1, inplace=True)
raw_data[raw_data['Context'] == raw_data['Context'].unique()[1]]

,Crime ID,Reported by,Falls within,Longitude,Latitude,LSOA name,Crime type,Context,year,month
55239,NaN,Greater Manchester Police,Greater Manchester Police,-2.444807,53.611509,Bolton 001A,Burglary,CrimeMapper has moved this record to a locatio...,2011,2
55250,NaN,Greater Manchester Police,Greater Manchester Police,-2.428078,53.622221,Bolton 001D,Other crime,CrimeMapper has moved this record to a locatio...,2011,2
55533,NaN,Greater Manchester Police,Greater Manchester Police,-2.388744,53.609646,Bolton 006D,Other crime,CrimeMapper has moved this record to a locatio...,2011,2
55546,NaN,Greater Manchester Police,Greater Manchester Police,-2.557438,53.594984,Bolton 007A,Burglary,CrimeMapper has moved this record to a locatio...,2011,2
55553,NaN,Greater Manchester Police,Greater Manchester Police,-2.557438,53.594984,Bolton 007A,Other crime,CrimeMapper has moved this record to a locatio...,2011,2
...,...,...,...,...,...,...,...,...,...,...
116263,NaN,Greater Manchester Police,Greater Manchester Police,-2.587383,53.478332,Wigan 038D,Other crime,CrimeMapper has moved this record to a locatio...,2011,3
116391,NaN,Greater Manchester Police,Greater Manchester Police,-2.565764,53.470625,Wigan 040E,Burglary,CrimeMapper has moved this record to a locatio...,2011,3
116392,NaN,Greater Manchester Police,Greater Manchester Police,-2.565764,53.470625,Wigan 040E,Burglary,CrimeMapper has moved this record to a locatio...,2011,3
116393,NaN,Greater Manchester Police,Greater Manchester Police,-2.565764,53.470625,Wigan 040E,Vehicle crime,CrimeMapper has moved this record to a locatio...,2011,3


In [10]:
raw_data[raw_data['Context'] == raw_data['Context'].unique()[2]]

,Crime ID,Reported by,Falls within,Longitude,Latitude,LSOA name,Crime type,Context,year,month
117201,b85812176163f5fa570ff3b8f5390a8e24d1849d3302ec...,Greater Manchester Police,Greater Manchester Police,-2.441435,53.592565,Bolton 011C,Other crime,This record has been moved to a location that ...,2011,4
117811,042a52d1d054f6416c36533df2ffd55b1732e8c76684b3...,Greater Manchester Police,Greater Manchester Police,-2.428325,53.576461,Bolton 016D,Violent crime,This record has been moved to a location that ...,2011,4
117851,e2de35589f6ba71723926e63f038cd106fa28957b735a5...,Greater Manchester Police,Greater Manchester Police,-2.428325,53.576461,Bolton 016D,Other crime,This record has been moved to a location that ...,2011,4
118748,0203ab3e20f2d9f4dd3c629d918363a38c048e82422c49...,Greater Manchester Police,Greater Manchester Police,-2.450872,53.567911,Bolton 023A,Other crime,This record has been moved to a location that ...,2011,4
119139,93bf84b206e3a8fafab2773e723917cbf52bcbc0681a16...,Greater Manchester Police,Greater Manchester Police,-2.407093,53.554666,Bolton 027C,Vehicle crime,This record has been moved to a location that ...,2011,4
...,...,...,...,...,...,...,...,...,...,...
428920,08a561787641217e9c14f977b5d0d7ffb630d2fbec206c...,Greater Manchester Police,Greater Manchester Police,-2.646118,53.545757,Wigan 015A,Criminal damage and arson,This record has been moved to a location that ...,2012,2
429185,a3c1f1acc5f904754063b7cae26a6f0a63f324fce8f7a5...,Greater Manchester Police,Greater Manchester Police,-2.665387,53.502291,Wigan 018G,Other theft,This record has been moved to a location that ...,2012,2
429753,b1d9fedbbe160639023f0b06ed5ed8fa0bbd3651205ff6...,Greater Manchester Police,Greater Manchester Police,-2.452878,53.494171,Wigan 029B,Burglary,This record has been moved to a location that ...,2012,2
429931,6de59d05d78163606e9c383d0893788bf76e3fc45c5bc2...,Greater Manchester Police,Greater Manchester Police,-2.492890,53.500008,Wigan 031A,Vehicle crime,This record has been moved to a location that ...,2012,2


In [11]:
# only keep non context values
raw_data = raw_data[raw_data['Context'].isna()].copy()

# now we can drop the Context column, as it doesn't hold anymore information
raw_data.drop('Context', axis=1, inplace=True)

# after some consideration, we decided to drop the crime ID column for now, it can easily be put back in if we want to change our approach
raw_data.drop('Crime ID', axis=1, inplace=True)

In [12]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(raw_data, test_size=0.2, random_state=15)

#### DO NOT CHANGE RANDOM STATE
#### use train set from now on, do not touch test_set, do not plot test_set, completely ignore it!!!!

# notes to investigate, do we need to stratify the data
# possible strata are, crime type and LSAO name
# set up a validation set, maybe cross validation

# it is easiest if you just use df_police from now on, it makes it easier to reload the initial train set, if you want to assign it to 
# extra vairables, make sure to use .copy() so you don't get errors/slow down your program
df_police = train_set.copy()
# df_police

## Average Annual Gross Income

In [13]:
# Get a set of all of the areas not in the Greater Manchester Area

df_police['Borough'] = df_police['LSOA name'].str[:-5] # Add first part of LSOA name as borough in the dataframe

lst_boroughs = ["Manchester", "Salford", "Bolton", "Bury", "Oldham", "Rochdale", "Stockport", 
                "Tameside", "Trafford", "Wigan"] # List contains all the boroughs of the Greater Manchester Area

lst_boroughs_in_df = df_police['Borough'].unique() # List contains all unique boroughs in the df_police dataframe

set_incorrect_boroughs = set(lst_boroughs_in_df) - set(lst_boroughs) # Set contains all areas that are in the dataframe but are not in the Greater Manchester Area

set_incorrect_boroughs

{'Blackburn with Darwen',
 'Calderdale',
 'Cheshire East',
 'Chorley',
 'High Peak',
 'Rossendale',
 'St. Helens',
 'Warrington',
 'West Lancashire',
 nan}

In [14]:
df_police['Falls within'].unique()

array(['Greater Manchester Police'], dtype=object)

In [15]:
df_police['Reported by'].unique()

array(['Greater Manchester Police'], dtype=object)

In [16]:
# Remove areas that don't fall within the Greater Manchester Area

for borough in set_incorrect_boroughs:
    df_police = df_police[df_police['Borough'] != borough]

df_police

,Reported by,Falls within,Longitude,Latitude,LSOA name,Crime type,year,month,Borough
291415,Greater Manchester Police,Greater Manchester Police,-2.156747,53.411334,Stockport 014B,Anti-social behaviour,2011,9,Stockport
66915,Greater Manchester Police,Greater Manchester Police,-2.242156,53.475729,Manchester 060A,Anti-social behaviour,2011,2,Manchester
2029897,Greater Manchester Police,Greater Manchester Police,-2.066454,53.478176,Tameside 015B,Anti-social behaviour,2016,12,Tameside
928577,Greater Manchester Police,Greater Manchester Police,-2.346607,53.391886,Trafford 024B,Vehicle crime,2013,9,Trafford
289092,Greater Manchester Police,Greater Manchester Police,-2.288561,53.491319,Salford 017B,Anti-social behaviour,2011,9,Salford
...,...,...,...,...,...,...,...,...,...
2806973,Greater Manchester Police,Greater Manchester Police,-2.308929,53.448510,Trafford 011D,Violence and sexual offences,2018,11,Trafford
2301899,Greater Manchester Police,Greater Manchester Police,-2.317831,53.493312,Salford 021B,Other theft,2017,8,Salford
799954,Greater Manchester Police,Greater Manchester Police,-2.263660,53.425482,Manchester 042D,Anti-social behaviour,2013,5,Manchester
451643,Greater Manchester Police,Greater Manchester Police,-2.069220,53.363858,Stockport 038C,Violent crime,2012,3,Stockport


In [17]:
# Get Median Annual Gross Pay data

df_pay = pd.read_excel('AnnualPayGrossManchester.xlsx')
df_pay = df_pay.set_index('Region')
df_pay

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Region,,,,,,,,,,,
Greater Manchester MC,29761,29339,25095,25292,25116,25660,26178,27004,27312,28000,29257
Bolton,21736,22457,21693,22801,22083,23301,24643,24140,24787,25662,25412
Bury,24014,23701,23752,23676,24442,25015,25404,23964,26383,25412,26752
Manchester,26542,26261,27113,27894,28283,27418,28547,29947,29997,30801,31623
Oldham,22132,22033,20933,20798,21937,22694,22992,22708,25422,25168,27396
Rochdale,20736,21581,21847,22629,21580,22222,22880,22655,23528,25100,25993
Salford,23710,25431,26180,26801,26564,27148,26815,28028,27932,28947,32366
Stockport,25038,26709,27222,27854,25630,26198,27036,27150,27525,29062,29132
Tameside,20978,21142,23058,21458,21438,21735,22924,24316,23058,23569,24148


In [18]:
# Get unique years in the df_police

df_police = df_police.dropna() # Drop row with a NaN value for Borough
lst_years = df_police['year'].unique() # List contains all unique years in df_police dataframe
lst_years

array([2011, 2016, 2013, 2018, 2012, 2014, 2015, 2017, 2019, 2010],
      dtype=int64)

In [19]:
# Add Median Annual Gross Pay to rows dependent of borough and year

df = pd.DataFrame() # Create empty dataframe to add agp to all combinations of boroughs and years seperately

for borough in lst_boroughs:
    df_borough = df_police.copy()
    df_borough = df_borough[df_borough['Borough'] == borough]
    for year in lst_years:
        df_year = df_borough.copy()
        df_year = df_year[df_year['year'] == year]
        df_year['magp'] = dict(df_pay)[year]['  ' + borough]
        df = pd.concat([df, df_year])   

In [20]:
# Replace df_police by the dataframe with the agp added

# df = df.sort_index()
df_police = df
df_police

,Reported by,Falls within,Longitude,Latitude,LSOA name,Crime type,year,month,Borough,magp
66915,Greater Manchester Police,Greater Manchester Police,-2.242156,53.475729,Manchester 060A,Anti-social behaviour,2011,2,Manchester,26261
89856,Greater Manchester Police,Greater Manchester Police,-2.234187,53.516894,Manchester 008E,Anti-social behaviour,2011,3,Manchester,26261
185957,Greater Manchester Police,Greater Manchester Police,-2.196252,53.486401,Manchester 015F,Other crime,2011,6,Manchester,26261
35149,Greater Manchester Police,Greater Manchester Police,-2.262393,53.402915,Manchester 046D,Burglary,2011,1,Manchester,26261
89758,Greater Manchester Police,Greater Manchester Police,-2.233141,53.507781,Manchester 008B,Other crime,2011,3,Manchester,26261
...,...,...,...,...,...,...,...,...,...,...
24367,Greater Manchester Police,Greater Manchester Police,-2.577786,53.535977,Wigan 013C,Violent crime,2010,12,Wigan,23396
24035,Greater Manchester Police,Greater Manchester Police,-2.622981,53.547741,Wigan 009C,Anti-social behaviour,2010,12,Wigan,23396
23650,Greater Manchester Police,Greater Manchester Police,-2.636478,53.579201,Wigan 004B,Anti-social behaviour,2010,12,Wigan,23396
25193,Greater Manchester Police,Greater Manchester Police,-2.533916,53.516137,Wigan 027A,Anti-social behaviour,2010,12,Wigan,23396


## Average Age

In [21]:
def get_avg_age_per_year(file_name):
    
    df = pd.read_excel(file_name)
    
    lst = []
    lst.append("LSOA Name")
    for i in range(90):
        lst.append(i)
    lst.append("90+")
    
    df = df[lst]
    
    df['Borough'] = df['LSOA Name'].str[:-5]
    lst = ["Manchester", "Salford", "Bolton", "Bury", "Oldham", "Rochdale", "Stockport", "Tameside", "Trafford", "Wigan"]
    lst2 = df['Borough'].unique()
    s1 = set(lst2) - set(lst)
    
    for i in s1:
        df = df[df['Borough'] != i]
    
    df = df.dropna()
    
    df = df.set_index('LSOA Name')
    df = df.drop(columns=["Borough"])
    df = df.rename(columns={'90+': 90})
    
    dic = dict()
    for lsoa in df.index.unique():
        dic[lsoa] = (df.T[lsoa] * df.T.index).sum() / df.T[lsoa].sum()
        
    df['avg'] = pd.Series(dic)
    df = df['avg']
    output = dict(df)
    
    return output

In [22]:
def add_avg_age_per_year(files):
    
    lst_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
    df_return = pd.DataFrame()
    
    for year, file in zip(lst_years, files):
        df = df_police[df_police['year'] == year]
        df['avg age'] = df['LSOA name'].map(get_avg_age_per_year(file))
        df_return = pd.concat([df_return, df])
        
    return df_return

In [23]:
lst_files = ['age_2012.xlsx', 'age_2012.xlsx', 'age_2013.xlsx', 'age_2014.xlsx', 'age_2015.xlsx', 'age_2016.xlsx',
             'age_2017.xlsx', 'age_2018.xlsx', 'age_2019.xlsx', 'age_2020.xlsx']
df = add_avg_age_per_year(lst_files)
# File for 2011 uses age groups and can therefore not be used to calculate an accurate average
# I suggest using the 2012 data
# Is it correct that we don't use data after 2019?

C:\Users\Koen\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\Koen\AppData\Local\Temp/ipykernel_8644/2636656142.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg age'] = df['LSOA name'].map(get_avg_age_per_year(file))
C:\Users\Koen\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\Koen\AppData\Local\Temp/ipykernel_8644/2636656142.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [24]:
df

,Reported by,Falls within,Longitude,Latitude,LSOA name,Crime type,year,month,Borough,magp,avg age
66915,Greater Manchester Police,Greater Manchester Police,-2.242156,53.475729,Manchester 060A,Anti-social behaviour,2011,2,Manchester,26261,29.175554
89856,Greater Manchester Police,Greater Manchester Police,-2.234187,53.516894,Manchester 008E,Anti-social behaviour,2011,3,Manchester,26261,33.990244
185957,Greater Manchester Police,Greater Manchester Police,-2.196252,53.486401,Manchester 015F,Other crime,2011,6,Manchester,26261,30.423439
35149,Greater Manchester Police,Greater Manchester Police,-2.262393,53.402915,Manchester 046D,Burglary,2011,1,Manchester,26261,42.462168
89758,Greater Manchester Police,Greater Manchester Police,-2.233141,53.507781,Manchester 008B,Other crime,2011,3,Manchester,26261,31.757060
...,...,...,...,...,...,...,...,...,...,...,...
2900810,Greater Manchester Police,Greater Manchester Police,-2.500789,53.528109,Wigan 017A,Public order,2019,2,Wigan,25010,39.647894
2968123,Greater Manchester Police,Greater Manchester Police,-2.463762,53.503945,Wigan 028A,Anti-social behaviour,2019,4,Wigan,25010,36.090274
2901854,Greater Manchester Police,Greater Manchester Police,-2.482098,53.505562,Wigan 031A,Vehicle crime,2019,2,Wigan,25010,35.272094
2900012,Greater Manchester Police,Greater Manchester Police,-2.624735,53.543831,Wigan 009A,Anti-social behaviour,2019,2,Wigan,25010,42.187842
